# Etiquetar las imagenes

In [64]:
import openai
from openai import OpenAI
from PIL import Image
import requests
import io
import base64
from io import BytesIO
import csv
import os



# Carga el cliente con su KEY

In [65]:
def load_api_key(file_path):
    try:
        with open(file_path, "r") as file:
            return file.read().strip()
    except Exception as e:
        raise ValueError(f"Error loading API key from {file_path}: {e}")

client = openai.Client(api_key=load_api_key('./key.txt'))

In [ ]:


def analyze_image(image_path):
    # Load the image
    image = Image.open(image_path)
    
    # Add a prompt to ask the model about the image characteristics
    prompt = ("Analyze the person in the image and label them with the following features:\n"
              "1. (0) short hair, (1) long hair\n"
              "2. (0) wears no glasses, (1) wears glasses\n"
              "3. (0) has no beard, (1) has beard\n"
              "4. (0) does not show teeth, (1) shows teeth\n\n"
              "Return the result as an array of 4 values in the format: [hair, glasses, beard, teeth].")

    # Convert the image to a format compatible with OpenAI API (if needed)
    buffered = io.BytesIO()
    image.save(buffered, format="PNG")
    image_bytes = buffered.getvalue()

    # Use OpenAI's API to analyze the image
    response = openai.Image.create_edit(
        prompt=prompt,
        n=1,
        size="1024x1024",
        image=image_bytes,
    )

    # Interpret the response (mock response since this capability may not exist directly)
    labels = [0, 1, 0, 1]  # Example mock response
    return labels

# Analyze multiple images
image_paths = ['Dataset P2/Celebrity Faces Dataset/Angelina Jolie/001_fe3347c0.jpg']  # Replace with your actual image paths
results = []

for image_path in image_paths:
    result = analyze_image(image_path)
    results.append(result)

print("Analysis Results:", results)


In [15]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What are in these images? Is there any difference between them?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)
print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The images you provided are identical, showing a natural landscape with a wooden pathway crossing through a grassy field under a blue sky with clouds. Since they are the same, there is no difference between them.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))


In [76]:
# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "tempImages/010_2f9c83bc.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "Im giving you an image of a person. Analyze the person in the image and label them with the following features:\n"
              "1. (0) has short hair, (1) has long hair, (2) you are not sure\n"
              "2. (0) wears no glasses, (1) wears glasses, (2) you are not sure\n"
              "3. (0) has no beard, (1) has a beard, (2) you are not sure\n"
              "4. (0) has the mouth closed, (1) has the mouth open, (2) you are not sure\n\n"
              "Return the result as an array of 4 values in the format: [hair, glasses, beard, teeth]. Return only the array and dont say anything else.",
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}",
            "detail": "high"
          },
        },
      ],
    }
  ],
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='[0, 0, 1, 0]', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))


In [28]:
# Step 1: Decode the base64 string
image_data = base64.b64decode(base64_image)

# Step 2: Open the image from the decoded bytes
image = Image.open(BytesIO(image_data))

# Step 3: Display the image using matplotlib
image.show()

# Itera sobre un directorio para devolver en forma de csv las etiquetas de las imagenes

In [79]:
results = []

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def analyze_image(image_path):
    base64_image = encode_image(image_path)
    
    # Make the API call
    try:
        response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": "Im giving you an image of a person. Analyze the person in the image and label them with the following features:\n"
                    "1. (0) has short hair, (1) has long hair, (2) you are not sure\n"
                    "2. (0) wears no glasses, (1) wears glasses, (2) you are not sure\n"
                    "3. (0) has no beard, (1) has a beard, (2) you are not sure\n"
                    "4. (0) has the mouth closed, (1) has the mouth open, (2) you are not sure\n"
                    "5. (0) is a male, (1) is a female, (2) you are not sure\n\n"
                    "Return the result as an array of 4 values in the format: [hair, glasses, beard, teeth]. Return only the array and dont say anything else.",
                },
                {
                "type": "image_url",
                "image_url": {
                    "url":  f"data:image/jpeg;base64,{base64_image}",
                    "detail": "high"
                },
                },
            ],
            }
        ],
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error analyzing image {image_path}: {e}")
        return None
  
# Function to process all images in a folder
def process_folder(folder_path, output_csv):
    results = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.lower().endswith(("jpg", "jpeg", "png")):  # Add more extensions if needed
                # Normalize the path to use forward slashes
                image_path = os.path.join(root, file)
                normalized_path = image_path.replace("\\", "/")
                print(f"Processing {normalized_path}...")
                analysis = analyze_image(normalized_path)
                if analysis:
                    results.append({"image": normalized_path, "analysis": analysis})
    
    # Save the results to a CSV file
    with open(output_csv, mode="w", newline="", encoding="utf-8") as csv_file:
        fieldnames = ["image", "analysis"]
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)

    print(f"Results saved to {output_csv}")

In [80]:
# Specify the folder containing images and the output CSV file
folder_path = "tempImages"
output_csv = "./tempImagesLabels.csv"

# Process the folder
process_folder(folder_path, output_csv)

Processing tempImages/002_24fab375.jpg...
Processing tempImages/010_18d7b27c.jpg...
Processing tempImages/010_2f9c83bc.jpg...
Processing tempImages/020_8e6bdc45.jpg...
Processing tempImages/027_3290d1bc.jpg...
Processing tempImages/071_d75194ff.jpg...
Processing tempImages/084_31281f33.jpg...
Processing tempImages/085_69f8d759.jpg...
Processing tempImages/085_97ab4600.jpg...
Processing tempImages/093_82a4677f.jpg...
Results saved to ./tempImagesLabels.csv
